In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import odeint,LSODA,BDF,solve_ivp
from scipy.linalg import eig
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RectWing.Models as rwm
import multiprocessing as mp

me.mechanics_printing()

In [7]:
def Model_Eigen_WT(model_num,b_modes,t_modes):
    dataset_name = 'WT'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 1.225
    p.s_t.value = 1.345
    p.c.value = 0.15
    p.EI.value = 61
    p.GJ.value = 200
    p.rho_t.value = 10
    p.e_0.value = 0
    p.e_1.value = 0
    p.ratio_fwt.value = 0.2
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    
    vars_ls =[]
    vars_ls.append((p.V,[10,15,20]))
    vars_ls.append((p.alpha_r,np.deg2rad(np.linspace(-5,15,21))))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.c_dmax,[0,1]))
    vars_ls.append((p.k_fwt,[0,1e5]))
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    
    variables = [k for k,v in vars_ls]
    sm_mini = sm.msubs(p.GetSubs(0,p.fp,ignore=variables))
    L = me.msubs((sm_mini.ExtForces.Q()[0]),{i:0 for i in p.qd})
    L_func = sym.lambdify((p.q,variables),L)
    
    flutdf = rw.fixed_point_finder(p,sm,vars_ls,True,additional_func={'Lift':L_func})
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'fp_data_{b_modes}B{t_modes}T-M{model_num}_{dataset_name}.pkl')

In [8]:
def Model_Eigen_HALE(model_num,b_modes,t_modes):
    dataset_name = 'HALE'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 0.0889
    p.s_t.value = 16
    p.c.value = 1
    p.EI.value = 2e4
    p.GJ.value = 1e4
    p.rho_t.value = 0.75
    p.e_0.value = 0.25
    p.e_1.value = 0
    
    p.ratio_fwt.value = 0.2
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    
    vars_ls =[]
    vars_ls.append((p.V,[15,20,25]))
    vars_ls.append((p.alpha_r,np.deg2rad(np.linspace(-5,15,21))))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.c_dmax,[0,1]))
    vars_ls.append((p.k_fwt,[0,1e6]))
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    
    variables = [k for k,v in vars_ls]
    sm_mini = sm.msubs(p.GetSubs(0,p.fp,ignore=variables))
    L = me.msubs((sm_mini.ExtForces.Q()[0]),{i:0 for i in p.qd})
    L_func = sym.lambdify((p.q,variables),L)
    
    flutdf = rw.fixed_point_finder(p,sm,vars_ls,True,additional_func={'Lift':L_func})
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'fp_data_{b_modes}B{t_modes}T-M{model_num}_{dataset_name}.pkl')

In [9]:
def Model_Eigen_JEC(model_num,b_modes,t_modes):
    dataset_name = 'JEC'
    p = rw.base_params(b_modes + t_modes + 2)
    p.rho.value = 1.225
    p.s_t.value = 12
    p.c.value = 1.8
    p.EI.value = 9.77e6
    p.GJ.value = 0.99e6
    p.rho_t.value = 19.53
    p.e_0.value = 0.08
    p.e_1.value = 0  
    p.ratio_fwt.value = 0.2
    
    p.alpha_s.value = np.deg2rad(15)
    p.eta.value = [1]*p.qs
    p.m_ac.value = 1e6
    p.k_ac.value = 1e6
    p.k_fwt.value = 0
    
    vars_ls =[] 
    vars_ls.append((p.V,[30,40,50]))
    vars_ls.append((p.alpha_r,np.deg2rad(np.linspace(-5,15,21))))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))
    vars_ls.append((p.c_dmax,[0,1]))
    vars_ls.append((p.k_fwt,[0,1e7]))
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    
    print(f'M{model_num}-{dataset_name}: Started')
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    
    variables = [k for k,v in vars_ls]
    sm_mini = sm.msubs(p.GetSubs(0,p.fp,ignore=variables))
    L = me.msubs((sm_mini.ExtForces.Q()[0]),{i:0 for i in p.qd})
    L_func = sym.lambdify((p.q,variables),L)
    
    flutdf = rw.fixed_point_finder(p,sm,vars_ls,True,additional_func={'Lift':L_func})
    print(f'M{model_num}-{dataset_name}: Complete')
    flutdf.to_pickle(f'fp_data_{b_modes}B{t_modes}T-M{model_num}_{dataset_name}.pkl')

In [10]:
b_modes = 3
t_modes = 3
pool = mp.Pool(mp.cpu_count())

for k in [0,1,2,3,4,5]:
    pool.apply_async(Model_Eigen_JEC,args=(k,b_modes,t_modes))
    pool.apply_async(Model_Eigen_HALE,args=(k,b_modes,t_modes))
    pool.apply_async(Model_Eigen_WT,args=(k,b_modes,t_modes))
pool.close()
pool.join()

M0-HALE: Started
M1-JEC: Started
M2-JEC: Started
M1-WT: Started
M1-HALE: Started
M0-JEC: Started
M0-WT: Started
M2-HALE: Started
M2-JEC: Complete
M2-WT: Started
M2-WT: Complete
M3-JEC: Started
M1-WT: Complete
M3-HALE: Started
M1-JEC: Complete
M3-WT: Started
M2-HALE: Complete
M4-JEC: Started
M4-JEC: Complete
M4-HALE: Started
M3-JEC: Complete
M4-WT: Started
M0-JEC: Complete
M5-JEC: Started
M0-WT: Complete
M5-HALE: Started
M3-WT: Complete
M5-WT: Started
M1-HALE: Complete
M4-WT: Complete
M5-JEC: Complete
M0-HALE: Complete
M3-HALE: Complete
M5-WT: Complete
M4-HALE: Complete
M5-HALE: Complete


In [3]:
%%time
b_modes = 3
t_modes = 3
Model_Eigen_WT(5,3,3)

M5-WT: Started
M5-WT: Complete
CPU times: user 1min 5s, sys: 480 ms, total: 1min 5s
Wall time: 1min 6s


In [ ]:
df = pd.read_pickle(f'fp_data_{b_modes}B{t_modes}T-M5_WT.pkl')
df.head(5)